# Neuron networks from scratch in Python
References: http://103.203.175.90:81/fdScript/RootOfEBooks/E%20Book%20collection%20-%202024%20-%20G/CSE%20%20IT%20AIDS%20ML/Neural%20Network.pdf

In [48]:
import numpy as np
import nnfs
from nnfs.datasets import spiral_data
nnfs.init()

## Chapter 16: Binary Logistic Regression

### 16.1 Sigmoid Activation Function

In [49]:
class Sigmoid:
    def forward(self, inputs):
        self.inputs = inputs
        self.output = 1 / (1 + np.exp(-inputs))
    
    def backward(self, dvalues):
        self.dinputs = dvalues * (1 - self.output) * self.output

### 16.2 Binary Cross-Entropy Loss

In [50]:
class Loss:
    def calculate(self, y_pred, y_true):
        return np.mean(self.forward(y_pred, y_true))

class BinaryCrossentropy(Loss):
    def forward(self, y_pred, y_true):
        y_pred = np.clip(y_pred, 1e-7, 1 - 1e-7)

        sample_losses = - (y_true * np.log(y_pred) + (1 - y_true) * np.log(1 - y_pred))
        sample_losses = np.mean(sample_losses, axis=-1)

        return sample_losses

    def backward(self, dvalues, y_true):
        samples = len(dvalues)
        outputs = len(dvalues[0])

        dvalues = np.clip(dvalues, 1e-7, 1 - 1e-7)
        self.dinputs = -(y_true / dvalues - (1 - y_true) / (1 - dvalues)) / outputs / samples

In [ ]:
# Implement

class Layer_Dense:
    def __init__(self, n_inputs, n_neurons, weight_regularizer_l1=0., weight_regularizer_l2=0.,
                 bias_regularizer_l1=0., bias_regularizer_l2=0.):
        self.weights = 0.01 * np.random.randn(n_inputs, n_neurons)
        self.biases = np.zeros((1, n_neurons))

        self.weight_regularizer_l1 = weight_regularizer_l1
        self.weight_regularizer_l2 = weight_regularizer_l2
        self.bias_regularizer_l1 = bias_regularizer_l1
        self.bias_regularizer_l2 = bias_regularizer_l2

    def forward(self, inputs):
        self.inputs = inputs
        self.output = np.dot(inputs, self.weights) + self.biases
    
    def backward(self, dvalues):
        self.dweights = np.dot(self.inputs.T, dvalues)
        self.dbiases = np.sum(dvalues, axis=0, keepdims=True)
        self.dinputs = np.dot(dvalues, self.weights.T)

        if self.weight_regularizer_l1 > 0:
            dL1 = np.ones_like(self.weights)
            dL1[self.weights < 0] = -1
            self.dweights += self.weight_regularizer_l1 * dL1 
        
        if self.weight_regularizer_l2 > 0:
            self.dweights += 2 * self.weight_regularizer_l2 * self.weights
        
        if self.bias_regularizer_l1 > 0:
            dL1 = np.ones_like(self.biases)
            dL1[self.biases < 0] = -1
            self.dbiases += self.bias_regularizer_l1 * dL1
        
        if self.bias_regularizer_l2 > 0:
            self.dbiases += 2 * self.bias_regularizer_l2 * self.biases

class Layer_Dropout:
    def __init__(self, dropout_rate):
        self.rate = 1 - dropout_rate
    
    def forward(self, inputs):
        self.inputs = inputs
        self.binary_mask = np.random.binomial(1, self.rate, inputs.shape) / self.rate
        self.output = inputs * self.binary_mask
    
    def backward(self, dvalues):
        self.dinputs = dvalues * self.binary_mask

class Activation_ReLU:
    def forward(self, inputs):
        self.inputs = inputs
        self.output = np.maximum(0, inputs)
    
    def backward(self, dvalues):
        self.dinputs = dvalues.copy()
        self.dinputs[self.inputs <= 0] = 0

class Activation_Softmax:
    def forward(self, inputs):
        exp_val = np.exp(inputs - np.max(inputs, axis=1, keepdims=True))
        self.output = exp_val / np.sum(exp_val, axis=1, keepdims=True)
    
    def backward(self, dvalues):

        self.dinputs = np.empty_like(dvalues)

        for idx, (single_output, single_dvalue) in enumerate(zip(self.output, dvalues)):
            single_output = np.reshape(single_output, (-1, 1))
            jacobian_matrix = np.diagflat(single_output) - np.dot(single_output, single_output.T)
            self.dinputs[idx] = np.dot(jacobian_matrix, single_dvalue)

class Activation_Sigmoid:
    def forward(self, inputs):
        self.inputs = inputs
        self.output = 1 / (1 + np.exp(- inputs))
    
    def backward(self, dvalues):
        self.dinputs = dvalues * (1 - self.output) * self.output

class Optimizer_SGD:
    def __init__(self, learning_rate=1., decay=0., momentum=0.):
        self.learning_rate = learning_rate
        self.current_learning_rate = learning_rate
        self.decay = decay
        self.momentum = momentum
        self.iterations = 1
    
    def pre_update_params(self):
        if self.decay:
            self.current_learning_rate = self.learning_rate / (1. + self.decay * self.iterations)
    
    def update_params(self, layer):
        if self.momentum:
            if not hasattr(layer, 'weight_momentums'):
                layer.weight_momentums = np.zeros_like(layer.weights)
                layer.bias_momentums = np.zeros_like(layer.biases)
            
            weights_update = self.momentum * layer.weight_momentums + self.current_learning_rate * layer.dweights
            layer.weight_momentums = weights_update
            layer.weights -= weights_update  


            biases_update = self.momentum * layer.bias_momentums + self.current_learning_rate * layer.dbiases
            layer.bias_momentums = biases_update
            layer.biases -= biases_update  

        else:
            layer.weights -= self.current_learning_rate * layer.dweights
            layer.biases -= self.current_learning_rate * layer.dbiases
        
    def post_update_params(self):
        self.iterations += 1

class Loss:
    def regularization_loss(self, layer):
        reg_loss = 0

        if layer.weight_regularizer_l1 > 0:
            reg_loss +=  layer.weight_regularizer_l1 * np.sum(np.abs(layer.weights))
        
        if layer.bias_regularizer_l1 > 0:
            reg_loss +=  layer.bias_regularizer_l1 * np.sum(np.abs(layer.biases))

        if layer.weight_regularizer_l2 > 0:
            reg_loss +=  layer.weight_regularizer_l2 * np.sum(layer.weights ** 2)
        
        if layer.bias_regularizer_l2 > 0:
            reg_loss +=  layer.bias_regularizer_l2 * np.sum(layer.biases ** 2)

        return reg_loss

    def calculate(self, y_pred, y_true):
        return np.mean(self.forward(y_pred, y_true))

class Loss_BinaryCrossentropy(Loss):
    def forward(self, y_pred, y_true):
        y_pred = np.clip(y_pred, 1e-7, 1 - 1e-7)
        sample_loss = - (y_true * np.log(y_pred) + (1 - y_true) * np.log(1 - y_pred)) 

        return np.mean(sample_loss , axis=-1)

    def backward(self, dvalues, y_true):
        samples = len(dvalues)
        outputs = len(dvalues[0])

        dvalues = np.clip(dvalues, 1e-7, 1 - 1e-7)

        self.dinputs = - (y_true / dvalues - (1 - y_true) / (1 - dvalues)) / outputs / samples


In [52]:
X, y = spiral_data(samples=1000, classes=2)

y = y.reshape(-1, 1)

dense1 = Layer_Dense(2, 64, weight_regularizer_l2=5e-4, bias_regularizer_l2=5e-4)
activation1 = Activation_ReLU()
dense2 = Layer_Dense(64, 1)
activation2 = Activation_Sigmoid()
loss_function = Loss_BinaryCrossentropy()
optimizer = Optimizer_SGD(learning_rate=0.01, decay=5e-4, momentum=5e-4)

epochs = 10000

for epoch in range(epochs):
    dense1.forward(X)
    activation1.forward(dense1.output)
    dense2.forward(activation1.output)
    activation2.forward(dense2.output)
    
    data_loss = loss_function.calculate(activation2.output, y)
    reg_loss = loss_function.regularization_loss(dense1) + loss_function.regularization_loss(dense2)
    loss = data_loss + reg_loss

    predictions = (activation2.output > 0.5) * 1
    accuracy = np.mean(predictions == y)

    if not epoch % 100:
        print(f'epoch: {epoch}, ' +
              f'acc: {accuracy:.3f}, '+
              f'loss: {loss:.3f} (' +
              f'data_loss: {data_loss:.3f}, ' +
              f'reg_loss: {reg_loss:.3f}), ' +
              f'lr: {optimizer.current_learning_rate}')

    loss_function.backward(activation2.output, y)
    activation2.backward(loss_function.dinputs)
    dense2.backward(activation2.dinputs)
    activation1.backward(dense2.dinputs)
    dense1.backward(activation1.dinputs)

    optimizer.pre_update_params()
    optimizer.update_params(dense1)
    optimizer.update_params(dense2)
    optimizer.post_update_params()

epoch: 0, acc: 0.543, loss: 0.693 (data_loss: 0.693, reg_loss: 0.000), lr: 0.01
epoch: 100, acc: 0.579, loss: 0.693 (data_loss: 0.693, reg_loss: 0.000), lr: 0.009523809523809523
epoch: 200, acc: 0.592, loss: 0.693 (data_loss: 0.693, reg_loss: 0.000), lr: 0.00909090909090909
epoch: 300, acc: 0.605, loss: 0.693 (data_loss: 0.693, reg_loss: 0.000), lr: 0.008695652173913044
epoch: 400, acc: 0.609, loss: 0.693 (data_loss: 0.693, reg_loss: 0.000), lr: 0.008333333333333333
epoch: 500, acc: 0.610, loss: 0.693 (data_loss: 0.693, reg_loss: 0.000), lr: 0.008
epoch: 600, acc: 0.607, loss: 0.693 (data_loss: 0.693, reg_loss: 0.000), lr: 0.007692307692307692
epoch: 700, acc: 0.606, loss: 0.693 (data_loss: 0.693, reg_loss: 0.000), lr: 0.007407407407407407
epoch: 800, acc: 0.606, loss: 0.693 (data_loss: 0.693, reg_loss: 0.000), lr: 0.0071428571428571435
epoch: 900, acc: 0.609, loss: 0.693 (data_loss: 0.693, reg_loss: 0.000), lr: 0.006896551724137932
epoch: 1000, acc: 0.611, loss: 0.693 (data_loss: 0.69

In [53]:
X_test, y_test = spiral_data(samples=100, classes=2)

y_test = y_test.reshape(-1, 1)


dense1.forward(X_test)
activation1.forward(dense1.output)
dense2.forward(activation1.output)
activation2.forward(dense2.output)

loss = loss_function.calculate(activation2.output, y_test)

predictions = (activation2.output > 0.5) * 1
accuracy = np.mean(predictions==y_test)

print(f'validation, acc: {accuracy:.3f}, loss: {loss:.3f}')

validation, acc: 0.595, loss: 0.692
